In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

In [2]:
# Definiendo las rutas a los datos de entrenamiento y prueba
train_dir = 'C:/Users/TAGOKG/Desktop/DatosV3/prueba'
test_dir = 'C:/Users/TAGOKG/Desktop/DatosV3/entrenamiento'

In [3]:
# Configurando los generadores de imágenes con ampliación de datos
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100, 100),
    batch_size=20,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(100, 100),
    batch_size=20,
    class_mode='categorical')

Found 5377 images belonging to 10 classes.
Found 5351 images belonging to 10 classes.


In [5]:
# Creando el modelo base de VGG16
base_model = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(100, 100, 3))

In [6]:
# Añadiendo nuevas capas
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)  # 10 frutas, entonces 10 clases


In [7]:
# Este es el modelo que vamos a entrenar
model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
# Primero, vamos a entrenar solo las capas superiores que acabamos de agregar
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])


In [9]:
# Entrenando el modelo con las nuevas capas
model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=20,
    validation_data=test_generator,
    validation_steps=50)

Epoch 1/20
100/100 [==============================] - 88s 865ms/step - loss: 1.1238 - accuracy: 0.6845 - val_loss: 0.5006 - val_accuracy: 0.8830
Epoch 2/20
100/100 [==============================] - 59s 587ms/step - loss: 0.4274 - accuracy: 0.9100 - val_loss: 0.4270 - val_accuracy: 0.8610
Epoch 3/20
100/100 [==============================] - 58s 584ms/step - loss: 0.2276 - accuracy: 0.9600 - val_loss: 0.3180 - val_accuracy: 0.9060
Epoch 4/20
100/100 [==============================] - 58s 586ms/step - loss: 0.1462 - accuracy: 0.9680 - val_loss: 0.3234 - val_accuracy: 0.9070
Epoch 5/20
100/100 [==============================] - 59s 589ms/step - loss: 0.1033 - accuracy: 0.9825 - val_loss: 0.3517 - val_accuracy: 0.9080
Epoch 6/20
100/100 [==============================] - 59s 590ms/step - loss: 0.0851 - accuracy: 0.9815 - val_loss: 0.4470 - val_accuracy: 0.9170
Epoch 7/20
100/100 [==============================] - 59s 586ms/step - loss: 0.0714 - accuracy: 0.9840 - val_loss: 0.3851 - val_ac

In [10]:
# A fino en las últimas capas de la red VGG16
for layer in model.layers[:15]:
    layer.trainable = False
for layer in model.layers[15:]:
    layer.trainable = True

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=1e-5),
              metrics=['accuracy'])


C:\Users\TAGOKG\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [11]:
loss, accuracy = model.evaluate(test_generator)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

268/268 [==============================] - 103s 381ms/step - loss: 0.6254 - accuracy: 0.9127
Loss: 0.6254414319992065
Accuracy: 0.9127265810966492


In [12]:
model.save('C:/Users/TAGOKG/Desktop/DatosV3/modelo1.h5')